In [35]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [36]:
import os

actions = [
    '나',
    '개학',
    '홍익대학교',
    '가다',
    '시험',
    '시작'
]

# Directory containing your .npy files
directory = 'dataset/seq/'

# Initialize an empty list to store the loaded data
data_list = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".npy"):
        file_path = os.path.join(directory, filename)
        loaded_data = np.load(file_path)
        data_list.append(loaded_data)

# Concatenate all loaded data into a single array
data = np.concatenate(data_list, axis=0)
data.shape

(136265, 30, 100)

In [37]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(136265, 30, 99)
(136265,)


In [38]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(136265, 6)

In [39]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(122638, 30, 99) (122638, 6)
(13627, 30, 99) (13627, 6)


In [40]:
x_train.shape[1:3]

(30, 99)

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=x_train.shape[1:3]),
    LSTM(32, activation='relu', return_sequences=False, input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 30, 64)            41984     
                                                                 
 lstm_9 (LSTM)               (None, 32)                12416     
                                                                 
 dense_8 (Dense)             (None, 32)                1056      
                                                                 
 dense_9 (Dense)             (None, 6)                 198       
                                                                 
Total params: 55,654
Trainable params: 55,654
Non-trainable params: 0
_________________________________________________________________


In [42]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=120,
    callbacks=[
        ModelCheckpoint('models/model_r8.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=10, verbose=1, mode='auto')
    ]
)

Epoch 1/120
3833/3833 [==============================] - ETA: 0s - loss: 1.6822 - acc: 0.5558
Epoch 1: val_acc improved from -inf to 0.63095, saving model to models\model_r8.h5
3833/3833 [==============================] - 98s 24ms/step - loss: 1.6822 - acc: 0.5558 - val_loss: 0.9829 - val_acc: 0.6310 - lr: 0.0010
Epoch 2/120
3833/3833 [==============================] - ETA: 0s - loss: 1.0011 - acc: 0.6395
Epoch 2: val_acc improved from 0.63095 to 0.71512, saving model to models\model_r8.h5
3833/3833 [==============================] - 96s 25ms/step - loss: 1.0011 - acc: 0.6395 - val_loss: 0.8059 - val_acc: 0.7151 - lr: 0.0010
Epoch 3/120
3831/3833 [============================>.] - ETA: 0s - loss: 0.6872 - acc: 0.7479
Epoch 3: val_acc improved from 0.71512 to 0.78851, saving model to models\model_r8.h5
3833/3833 [==============================] - 89s 23ms/step - loss: 0.6872 - acc: 0.7479 - val_loss: 0.5872 - val_acc: 0.7885 - lr: 0.0010
Epoch 4/120
3831/3833 [==========================

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model_r8.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))